# This is going to be awesome

### Imports

In [1]:
import pandas as pd
import numpy as np

In [10]:
pre = pd.read_csv('data/pre.csv', delimiter='","')
pre

/var/folders/np/z3d7fsz132vckb3gj8c2j2c00000gn/T/ipykernel_1865/534607909.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  pre = pd.read_csv('data/pre.csv', delimiter='","')


,"""Timestamp,""""Participant ID (top-right corner of the screen)""","""How old are you?""","""Gender?""","""What describes you the best?""","""How comfortable are you in expressing emotions?""","""Have you ever played with the """"""""drummer bot"""""""" dB before?""","""How much prior experience do you have with interactive musical interfaces (e.g. sensor systems, digital control devices)?""","""How often do you use AI tools for creativity (e.g., writing, coding, music, fine arts, food recipes, etc.)?""","""Please indicate if you have any of the following health problems."""""";"
0,"""2024/11/04 5:15:10 PM GMT+1,""""test""","""66""","""Prefer not to say""","""Student/semi-professional musician""","""3""","""Yes""","""4""","""4""","""anan"""""";"
1,"""2024/11/05 12:45:26 PM GMT+1,""""11""","""11""","""Prefer not to say""","""Student/semi-professional musician""","""0""","""No""","""0""","""0""","""Test"""""";"
2,"""2024/11/05 5:39:46 PM GMT+1,""""andsti-01""","""25""","""Male""","""Avid music listener""","""2""","""No""","""2""","""1""",""""""""";"
3,"""2024/11/05 5:49:00 PM GMT+1,""""andsti-01""","""25""","""Male""","""Avid music listener""","""2""","""No""","""2""","""1""",""""""""";"
4,"""2024/11/05 6:52:10 PM GMT+1,""""grup16""","""21""","""Female""","""Avid music listener""","""4""","""No""","""0""","""2""",""""""""";"
...,...,...,...,...,...,...,...,...,...
138,"""2024/11/11 12:15:01 PM GMT+1,""""6_4""","""24""","""Male""","""Avid music listener""","""3""","""No""","""1""","""4""",""""""""";"
139,"""2024/11/11 12:30:56 PM GMT+1,""""gr13-knh3""","""28""","""Male""","""Avid music listener""","""2""","""No""","""4""","""3""",""""""""";"
140,"""2024/11/11 12:33:52 PM GMT+1,""""8_testing4""","""30""","""Female""","""Professional musician""","""4""","""No""","""0""","""3""",""""""""";"
141,"""2024/11/11 1:22:26 PM GMT+1,""""6_5""","""25""","""Female""","""Averagely interested in music""","""3""","""No""","""1""","""3""",""""""""";"
